<a href="https://colab.research.google.com/github/mrtingalingling/MorvPAC/blob/main/EnDorphment_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Pyfhel & Initiate access

In [ ]:
!pip install Pyfhel

In [ ]:
import numpy as np
from Pyfhel import Pyfhel, PyCtxt

In [ ]:
!pip install PyGithub

In [ ]:
# Adding GitHub access
import requests
from github import Github

# Authentication is defined via github.Auth
from github import Auth

# using an access token
access_token = "github_pat_..."
auth = Auth.Token(access_token)

# Public Web Github
g = Github(auth=auth)
repo = g.get_repo("mrtingalingling/MorvPAC")

print(repo.name)
# To close connections after use
# g.close()

MorvPAC


## Encryption Context & Client Setup

"All these schemes are based on the hardness
of the Ring Learning With Errors (RLWE) problem, where noise is added during encryption and key generation to achieve the hardness properties. The noise grows as encrypted computations are performed, and the main functional parameter in all these schemes, the ciphertext modulus Q,
needs to be large enough to accommodate the noise growth, or a special bootstrapping procedure may be used to reset the noise and keep the value of Q relatively small." -- https://eprint.iacr.org/2021/204.pdf

In [ ]:
# Generate Pyfhel session
print(f"[Client] Initializing Pyfhel session and data...")
HE_client = Pyfhel()           # Creating empty Pyfhel object
HE_client.contextGen(scheme='bfv', n=2**14, t_bits=20) # Not going to think about this for now
                        # The n defines the number of plaintext slots.
                        # https://pyfhel.readthedocs.io/en/latest/_autosummary/Pyfhel.Pyfhel.html#Pyfhel.Pyfhel.contextGen
HE_client.keyGen()             # Key Generation: generates a pair of public/secret keys
HE_client.relinKeyGen()
HE_client.rotateKeyGen()

[Client] Initializing Pyfhel session and data...


## 1. Verified Users
After the user has logged in to the app, we would double check user's eligibility to vote through MFA:

1. Eligible voters would receive a voting verification code (VVC)
2. The VVC would be entered at the same time as the MFA


In [ ]:
## Create user
import random
import string
from collections import defaultdict

number_of_users = 100
user_dict = defaultdict(dict)
print(f"Assuming {number_of_users} users")

# initializing size of string
N = 7

for each_user in range(number_of_users):
  # Use the while loop to ensure user hash is unique
  while len(user_dict.keys()) <= each_user:
    # generating random strings as user hash
    res = ''.join(random.choices(string.ascii_uppercase + string.digits, k=N))
    if res not in user_dict.keys():
      # [random.randint(0, 5) for _ in range(number_of_choices)]
      user_dict[res] = random.randint(0, 1)

# print result
print("Number of Users generated: ", len(user_dict.keys()))
print("Number of Votes Yes: ", sum(user_dict.values()))
print("Sampled User Hash: ", list(user_dict.keys())[0])


Assuming 100 users
Number of Users generated:  100
Number of Votes Yes:  43
Sampled User Hash:  NYO3HQV


## 2. Encrypt Votes


In [ ]:
import time
# Generate and encrypt data
# result_arr = np.array(list(user_dict.values()))
# cx = HE_client.encrypt(result_arr)
# print(f"[Client] sending HE_client={HE_client} and cx={cx}")

def encryptResponse(HE, result: int) -> Pyfhel:
  return HE.encryptInt(np.array([result], dtype=np.int64))

cx_ls = []

for _user, _response in user_dict.items():
  cx = encryptResponse(HE_client, _response)
  time.sleep(0.005)
  # print(f"[Client] sending HE_client={HE_client} and cx={cx}")
  cx_ls.append(cx)

event = "Test"

In [ ]:
con_size, con_size_zstd   = HE_client.sizeof_context(),    HE_client.sizeof_context(compr_mode="zstd")
pk_size,  pk_size_zstd    = HE_client.sizeof_public_key(), HE_client.sizeof_public_key(compr_mode="zstd")
sk_size,  sk_size_zstd    = HE_client.sizeof_secret_key(), HE_client.sizeof_secret_key(compr_mode="zstd")
rotk_size,rotk_size_zstd  = HE_client.sizeof_rotate_key(), HE_client.sizeof_rotate_key(compr_mode="zstd")
rlk_size, rlk_size_zstd   = HE_client.sizeof_relin_key(),  HE_client.sizeof_relin_key(compr_mode="zstd")
c_size,   c_size_zstd     = cx.sizeof_ciphertext(),  cx.sizeof_ciphertext(compr_mode="zstd")
# alternatively, for ciphertext sizes you can use sys.getsizeof(c)

print("2. Checking size of serializable objects (with and without compression)")
print(f"   - context:    [ \"zstd\"  --> {con_size_zstd} | No compression --> {con_size}]")
print(f"   - public_key: [ \"zstd\"  --> {pk_size_zstd} | No compression --> {pk_size}]")
print(f"   - secret_key: [ \"zstd\"  --> {sk_size_zstd} | No compression --> {sk_size}]")
print(f"   - relin_key:  [ \"zstd\"  --> {rotk_size_zstd} | No compression --> {rotk_size}]")
print(f"   - rotate_key: [ \"zstd\"  --> {rlk_size_zstd} | No compression --> {rlk_size}]")
print(f"   - c:          [ \"zstd\"  --> {c_size_zstd} | No compression --> {c_size}]")

2. Checking size of serializable objects (with and without compression)
   - context:    [ "zstd"  --> 337 | No compression --> 273]
   - public_key: [ "zstd"  --> 2368625 | No compression --> 2359409]
   - secret_key: [ "zstd"  --> 1184344 | No compression --> 1179736]
   - relin_key:  [ "zstd"  --> 492805731 | No compression --> 490888200]
   - rotate_key: [ "zstd"  --> 18949067 | No compression --> 18875336]
   - c:          [ "zstd"  --> 2105457 | No compression --> 2097265]


## 3. Prep Data for uploading them to public server

In [ ]:
# Serializing data and public context information
s_context    = HE_client.to_bytes_context()
s_public_key = HE_client.to_bytes_public_key()
s_relin_key  = HE_client.to_bytes_relin_key()
s_rotate_key = HE_client.to_bytes_rotate_key()
s_cx         = cx.to_bytes()
s_secret_key = HE_client.to_bytes_secret_key()

print("Save all objects into byte-strings")
print(f"   - s_context: {s_context[:10]}...")
print(f"   - s_public_key: {s_public_key[:10]}...")
print(f"   - s_relin_key: {s_relin_key[:10]}...")
print(f"   - s_rotate_key: {s_rotate_key[:10]}...")
print(f"   - s_c: {s_cx[:10]}...")
print(f"   - s_secret_key: {s_secret_key[:10]}...")


Save all objects into byte-strings
   - s_context: b'\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00'...
   - s_public_key: b'^\xa1\x10\x04\x00\x02\x00\x00\xfeb'...
   - s_relin_key: b'^\xa1\x10\x04\x00\x02\x00\x00\xd9\x16'...
   - s_rotate_key: b'^\xa1\x10\x04\x00\x02\x00\x00b\xb2'...
   - s_c: b'^\xa1\x10\x04\x00\x00\x00\x00q\x00'...
   - s_secret_key: b'^\xa1\x10\x04\x00\x02\x00\x00\xbc\xb1'...


In [ ]:
!pip install pyairtable

In [ ]:
# Submit data to AirTable for audit
AIRTABLE_API_KEY = 'pat...'
from pyairtable import Api
api = Api(AIRTABLE_API_KEY)
table = api.table('appI6fmXeNf4ZYt3K', 'tbluVeNSYPOUKYxqZ')


In [ ]:
cx_filename = f"/{_user}_{event}_c.ctxt"
context_filename = f"/{_user}_{event}_context"

import pickle
pkls_pyfhel = pickle.dumps(HE_client)   # pickle.dump(HE, file) to dump in a file
pkls_ctxt   = pickle.dumps(cx)

print("5a. Pickling Pyfhel & PyCtxt objects.")
print(f"  - pkls_pyfhel: {pkls_pyfhel[:10]}...")
print(f"  - pkls_ctxt: {pkls_ctxt[:10]}...")


5a. Pickling Pyfhel & PyCtxt objects.
  - pkls_pyfhel: b'\x80\x04\x95\xba\x00\x00\x00\x00\x00\x00'...
  - pkls_ctxt: b'\x80\x04\x95\xd7\x00\x00\x00\x00\x00\x00'...


In [ ]:
# Check GitHub
# https://stackoverflow.com/questions/63427607/python-upload-files-directly-to-github-using-pygithub
all_files = []
contents = repo.get_contents("")
while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
        contents.extend(repo.get_contents(file_content.path))
    else:
        file = file_content
        all_files.append(str(file).replace('ContentFile(path="','').replace('")',''))

# Upload to github
def Upload2GitHub(repo, filename:str, content, gitDir:str="Results"):
  git_file = f'{gitDir}' + filename
  print(git_file)
  if git_file in all_files:
      print("Response already recorded")
  else:
      repo.create_file(git_file, "committing files", content, branch="main")
      print(git_file + ' CREATED')


In [ ]:
Upload2GitHub(repo, cx_filename, pkls_ctxt)
Upload2GitHub(repo, context_filename, pkls_pyfhel)


Results/QL3UQVX_Test_c.ctxt
Results/QL3UQVX_Test_c.ctxt CREATED
Results/QL3UQVX_Test_context
Results/QL3UQVX_Test_context CREATED


In [ ]:
table.create({"Hash": _user,
              "Context": "https://github.com/mrtingalingling/MorvPAC/blob/main/Results" + context_filename,
              "Response": "https://github.com/mrtingalingling/MorvPAC/blob/main/Results" + cx_filename,
              "Event": event
              })

{'id': 'recnIRpnh8Hm5O7d0',
 'createdTime': '2023-09-02T23:12:42.000Z',
 'fields': {'Hash': 'QL3UQVX',
  'Context': 'https://github.com/mrtingalingling/MorvPAC/blob/main/Results/QL3UQVX_Test_context',
  'Response': 'https://github.com/mrtingalingling/MorvPAC/blob/main/Results/QL3UQVX_Test_c.ctxt',
  'Event': 'Test',
  'Submission Time': '2023-09-02T23:12:42.000Z',
  'Modified Time': '2023-09-02T23:12:42.000Z'}}

## 4. Loaded and operate the data from the server

In [ ]:
# Load the objects, just call `pickle.loads`
HE_b = pickle.loads(pkls_pyfhel) # pickle.load(file) to load from file
cx_b = pickle.loads(pkls_ctxt)
print(f"[Server] received HE_server={HE_b} and cx={cx}")

ctxtSum = 0
for v in cx_ls:
  ctxtSum += v

print(f"Sum: {ctxtSum}")

[Server] received HE_server=<bfv Pyfhel obj at 0x79bbc98a5250, [pk:-, sk:-, rtk:-, rlk:-, contx(n=16384, t=786433, sec=128, qi=[48, 48, 48, 49, 49, 49, 49, 49, 49], scale=1.0, )]> and cx=<Pyfhel Ciphertext at 0x79bc1a1538d0, scheme=bfv, size=2/2, noiseBudget=361>
Sum: <Pyfhel Ciphertext at 0x79bbc991ba10, scheme=bfv, size=2/2, noiseBudget=358>


## 5. Validation

In [ ]:
# Load everything and quickly check if it works.
HE_b.from_bytes_public_key(s_public_key)
HE_b.from_bytes_secret_key(s_secret_key)
HE_b.from_bytes_relin_key(s_relin_key)
HE_b.from_bytes_rotate_key(s_rotate_key)
cx_b = PyCtxt(pyfhel=HE_b, bytestring=s_cx)
# p_b = PyPtxt(pyfhel=HE_b, bytestring=s_p)

print("Loading everything from bytestrings.")
# Some checks
assert HE_b.decryptInt(HE_b.encryptInt(np.array([42], dtype=np.int64)))[0]==42, "Incorrect encryption"
assert HE_b.decryptInt(cx_b)[0]==_response, "Incorrect decryption/ciphertext"
# assert HE_b.decodeInt(p_b)[0]==-1, "Incorrect decoding"
assert HE_b.decryptInt(cx_b >> 1)[1]==_response, "Incorrect Rotation"
c_relin = cx_b**2
~c_relin
assert c_relin.size()==2, "Incorrect relinearization"
print("  All checks passed! Loaded from bytestrings correctly")

# Decrypt the final result
resSum = HE_b.decryptInt(ctxtSum)

print("#. Decrypting result:")
print("     addition:       decrypt(ctxtSum) =  ", resSum[0])
assert resSum[0]==sum(user_dict.values()), "Incorrect addition"

for each_value in resSum:
  if each_value != 0:
    print(each_value)

Loading everything from bytestrings.
  All checks passed! Loaded from bytestrings correctly
#. Decrypting result:
     addition:       decrypt(ctxtSum) =   43
43


Every user would submit an encrypted vote (essentially, a +1 on every candidate they approve of, and a +0 on every candidate they do not approve of). A file comprised of ALL of these votes would amount to the election result.

Anyone could download that file, check that their vote is part of the file, then compute the sum of all the encrypted votes, which must amount to the published election results.

Results: https://airtable.com/appI6fmXeNf4ZYt3K/shrzYGBi3roRJFKQr